<a href="https://colab.research.google.com/github/chuan137/egohands/blob/master/grocery/train_category_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Note

### Update
* use image data generator to split validation and train data

### Training

* one fully connected layer
* softmax layer with regularizer, weight = 0.0001
* optimizer Adam, learning rate 0.001
* image size 96
* steps per epoch 200
* validation steps 50


## Prepare

In [0]:
!pip install --upgrade -q tensorflow-gpu==2.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 450kB 40.7MB/s 
     |████████████████████████████████| 3.8MB 30.1MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
2.0.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%set_env PROJECT=/content/drive/My Drive/Colab Notebooks/grocery
!cp "$PROJECT"/data/images.zip .
!unzip -q images.zip
!cp "$PROJECT"/data/*.csv .

env: PROJECT=/content/drive/My Drive/Colab Notebooks/grocery


## Load Data

In [0]:
classes = {
    'BEANS': 0,
    'CAKE': 1,
    'CANDY': 2,
    'CEREAL': 3,
    'CHIPS': 4,
    'CHOCOLATE': 5,
    'COFFEE': 6,
    'CORN': 7,
    'FISH': 8,
    'FLOUR': 9,
    'HONEY': 10,
    'JAM': 11,
    'JUICE': 12,
    'MILK': 13,
    'NUTS': 14,
    'OIL': 15,
    'PASTA': 16,
    'RICE': 17,
    'SODA': 18,
    'SPICES': 19,
    'SUGAR': 20,
    'TEA': 21,
    'TOMATO_SAUCE': 22,
    'VINEGAR': 23,
    'WATER': 24
    }

IMAGE_SIZE = 224
BATCH_SIZE = 32
CLASS_SIZE = 25
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    './images',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    subset='training'
)

val_generator = datagen.flow_from_directory(
    './images',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    subset='validation'
)


Found 3967 images belonging to 25 classes.
Found 980 images belonging to 25 classes.


In [0]:
import tensorflow as tf
from tensorflow.keras import regularizers

w_l2 = 0.0001
n_class = 25

# Pre-trained model with MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
# Freeze the pre-trained model weights
base_model.trainable = False
# Trainable classification head
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
fc_layer_1 = tf.keras.layers.Dense(1280, activation='relu')
# Prediction with L2 regularizer
prediction_layer = tf.keras.layers.Dense(
    n_class, 
    kernel_regularizer=regularizers.l2(w_l2), 
    activation='softmax')

# Layer classification head with feature detector
model = tf.keras.Sequential([
    base_model,
    maxpool_layer,
    fc_layer_1,
    prediction_layer
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_max_pooling2d_4 (Glob (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1280)              1639680   
_________________________________________________________________
dense_9 (Dense)              (None, 25)                32025     
Total params: 3,929,689
Trainable params: 1,671,705
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
learning_rate = 0.001
num_epochs = 50
steps_per_epoch = 100
val_steps = 100

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss='categorical_crossentropy',
              metrics=['accuracy']
)

# Fit
hist = model.fit_generator(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=num_epochs,
            validation_data=val_generator,
            validation_steps=val_steps)

Epoch 1/50
100/100 [==============================] - 59s 590ms/step - loss: 14.2949 - accuracy: 0.0794 - val_loss: 14.5230 - val_accuracy: 0.0863
Epoch 2/50
100/100 [==============================] - 59s 586ms/step - loss: 14.5140 - accuracy: 0.0919 - val_loss: 14.4534 - val_accuracy: 0.0882
Epoch 3/50
100/100 [==============================] - 58s 581ms/step - loss: 14.6964 - accuracy: 0.0844 - val_loss: 14.9551 - val_accuracy: 0.0730
Epoch 4/50
100/100 [==============================] - 58s 582ms/step - loss: 14.9727 - accuracy: 0.0713 - val_loss: 14.9278 - val_accuracy: 0.0749
Epoch 5/50
100/100 [==============================] - 59s 587ms/step - loss: 14.9126 - accuracy: 0.0750 - val_loss: 14.9045 - val_accuracy: 0.0752
Epoch 6/50
 99/100 [============================>.] - ETA: 0s - loss: 14.9002 - accuracy: 0.0758